In [16]:
#------------------------ ###
# Project QB
# Module : POC Encounter
# Extract all 'poc encounter' params from input text wrt excel and remove extracted params
# -----------------------###

import pandas as pd
from Modules.For_All.Text_Preprocessing import text_clean

In [17]:
#create Triggerword List
#--------#

df2 = pd.read_excel('Adhoc_QB_CH_POC_Encounter_Entities_31_10_2023.xlsx',sheet_name='Trigger_Words')
df2 = df2['TriggerWords']
trigger_words_list=[]
for word in df2:
    word= text_clean(word)
    if word not in trigger_words_list:
        trigger_words_list.append(word)

In [18]:
#-----------------------------------------------------
# Create 'POCENcounter_Params_Dict' with Preprocessing'
#-----------------------------------------------------
df_overall = pd.read_excel('Adhoc_QB_CH_POC_Encounter_Entities_31_10_2023.xlsx',sheet_name='Params_Sheet')
column_names= df_overall.columns                                               # Get all columns names

all_pocencounter_params_dict={}                                      
for column in column_names:
    df_column=df_overall[column]                                               # get all data from specifice column
    df_column= df_column.dropna()                                              # remove N/A field

    column_entity_list=[]
    for entity in df_column:
        clean_entity= text_clean(entity)                                       # clean code using module
        if clean_entity:
            column_entity_list.append(clean_entity)                            # append in list
    column_entity_list= sorted(column_entity_list, key=len, reverse=True)
    all_pocencounter_params_dict[column]= column_entity_list

In [19]:
#==========================================================
# Main Function: 'Params Extraction'
#==========================================================
def pocencounter_extraction(f_text):
    f_text= ' '+ f_text +' '
    field_status=False
        
    #--------------------------------------------------
    # set 'field_status' True if 'poc emcounter' trigger word mentioned in query
    #--------------------------------------------------
    for trigger_word in trigger_words_list:                         
        trigger_word= ' '+trigger_word+' '
        if trigger_word in f_text:
            field_status=True 
     

    #--------------------------------------------------
    # Extract params from text using excel_sheet_dict
    #--------------------------------------------------
    pocencounter_params_dict={}                                                   # 'final dict' to save result
    for field_name_key,param_field_list_value in all_pocencounter_params_dict.items():
        if field_name_key in ['POCENCOUNTER CODE','POCENCOUNTER DISPLAYNAME','POCECNOUNTER SOURCE']:
            if field_status == False:
                continue                                                   # skip current loop cycle

        extracted_param_list=[]
        for param in param_field_list_value:
            start_index = f_text.find(' '+str(param)+' ')                  # find the index of param in text
            if start_index != -1: 
                end_index = start_index + len(param)                       # index of the last occurrence of the string in the text
                param_name = f_text[start_index:end_index+1]               # pick words using start & end index
                param_name=" ".join(param_name.split())                    # remove extra spaces in selected data
                extracted_param_list.append(param_name)                    # append extracted param in list using current key,value of length_dict
                f_text= f_text.replace(' '+param+' ',' ')
        if extracted_param_list:                                           # check: if list is not empty
            try:
                pocencounter_params_dict[field_name_key].extend(extracted_param_list)
            except:
                pocencounter_params_dict[field_name_key]= extracted_param_list

    return [pocencounter_params_dict,f_text]

In [21]:

text= "Show me all the patients who had a provision of care encounter code and the poc source was externa ehr"
clean_text= text_clean(text)
result= pocencounter_extraction(clean_text)
print('\n',result[0])
print('\n',result[1])



 {'POCENCOUNTER CODE': ['provision of care encounter code'], 'POCECNOUNTER SOURCE': ['poc source']}

  show me all the patients who had a and the was externa ehr 
